# Multivariate association with MASH
This note book document the process of meta analysis using mash. 

## Input:
22 merged.vcf.gz file as output of the sumstat merger.

## Output
The output of association scan, and optionally the output from METAL, will undergo a reformmating process, and then fed into the MASH module to generate
1. MASH result in vcf format
2. Mixture prior for MvSuSIE_RSS

## Partitioning

In [ ]:
nohup sos run pipeline/sumstat_processing.ipynb processing \
    --sumstat_inv `ls /mnt/vast/hpc/csg/snuc_pseudo_bulk/eight_celltypes_analysis/output/data_intergration/TensorQTL/*norminal.cis_long_table.merged.vcf.gz`

## Extract effect
The stronges, the random, and the null signal will be extracted for each gene, and merged into 1 rds for downstream analysis. Only 6 phenotypes ( cell types) will be used because `ALL` phenotype the pooled of all other celltypes and there are too many non-overlap gene if `End` is not excluded, 1600 overlaped gene out of 13000 genes. Even with `End` excluded, there is only 6000 overlapped gene left.

### Example analysis unit

In [1]:
head /mnt/vast/hpc/csg/snuc_pseudo_bulk/eight_celltypes_analysis/MASH/output/ALL_Ast_End_Exc_Inh_Mic_OPC_Oli.merged_rds.list

HES4	/mnt/vast/hpc/csg/snuc_pseudo_bulk/eight_celltypes_analysis/MASH/output/RDS/ALL_Ast_End_Exc_Inh_Mic_OPC_Oli.log2cpm.bed.processed_phenotype.per_chrom_ALL_Ast_End_Exc_Inh_Mic_OPC_Oli.log2cpm.ALL_Ast_End_Exc_Inh_Mic_OPC_Oli.covariate.pca.resid.PEER.cov.chr1.norminal.cis_long_tableHES4.rds
KLHL17	/mnt/vast/hpc/csg/snuc_pseudo_bulk/eight_celltypes_analysis/MASH/output/RDS/ALL_Ast_End_Exc_Inh_Mic_OPC_Oli.log2cpm.bed.processed_phenotype.per_chrom_ALL_Ast_End_Exc_Inh_Mic_OPC_Oli.log2cpm.ALL_Ast_End_Exc_Inh_Mic_OPC_Oli.covariate.pca.resid.PEER.cov.chr1.norminal.cis_long_tableKLHL17.rds
NOC2L	/mnt/vast/hpc/csg/snuc_pseudo_bulk/eight_celltypes_analysis/MASH/output/RDS/ALL_Ast_End_Exc_Inh_Mic_OPC_Oli.log2cpm.bed.processed_phenotype.per_chrom_ALL_Ast_End_Exc_Inh_Mic_OPC_Oli.log2cpm.ALL_Ast_End_Exc_Inh_Mic_OPC_Oli.covariate.pca.resid.PEER.cov.chr1.norminal.cis_long_tableNOC2L.rds
LINC01128	/mnt/vast/hpc/csg/snuc_pseudo_bulk/eight_celltypes_analysis/MASH/output/RDS/ALL_Ast_End_Exc_Inh_Mic_OPC_O

In [ ]:
nohup sos run pipeline/Signal_Extraction.ipynb extract_effects     \
    --name  Ast_Exc_Inh_Mic_OPC_Oli    \
    --analysis_units   <(cat output/ALL_Ast_End_Exc_Inh_Mic_OPC_Oli.merged_rds.list | cut -f 2 ) \
    --exclude_condition 1,3  -s force -J 20 -q csg -c csg.yml  &

## Construct of MASH Model

In [ ]:
 nohup sos run pipeline/mashr.ipynb mash --container //mnt/vast/hpc/csg/containers/stephenslab.sif \
     --output_prefix Ast_Exc_Inh_Mic_OPC_Oli \
     --data output/Ast_Exc_Inh_Mic_OPC_Oli.rds\
     -s force -J 20 -q csg -c csg.yml --cwd MASH_6_celltypes --vhat mle &

## MASH Posterior
Run the following command in the same working directory as the previouse step

In [ ]:
 nohup sos run pipeline/mashr.ipynb posterior --container //mnt/vast/hpc/csg/containers/stephenslab.sif \
     --output_prefix Ast_Exc_Inh_Mic_OPC_Oli \
     --data <(cat output/ALL_Ast_End_Exc_Inh_Mic_OPC_Oli.merged_rds.list | cut -f 2 ) \
     -s force -J 200 -q csg -c csg.yml --cwd MASH_6_celltypes --vhat mle &

## Convert MASH result RDS to VCF
Running the follwoing code using the mash_output_list from the previouse step as input can covert all the mash RDS into vcf format

### Example analysis unit

In [2]:
head /mnt/mfs/statgen/xqtl_workflow_testing/module/MASH/mash_output_list

"/mnt/mfs/statgen/xqtl_workflow_testing/module/MASH/PCC_DLPFC_AC_ENSG00000000457.posterior.rds"
"/mnt/mfs/statgen/xqtl_workflow_testing/module/MASH/PCC_DLPFC_AC_ENSG00000000460.posterior.rds"
"/mnt/mfs/statgen/xqtl_workflow_testing/module/MASH/PCC_DLPFC_AC_ENSG00000000938.posterior.rds"
"/mnt/mfs/statgen/xqtl_workflow_testing/module/MASH/PCC_DLPFC_AC_ENSG00000000971.posterior.rds"
"/mnt/mfs/statgen/xqtl_workflow_testing/module/MASH/PCC_DLPFC_AC_ENSG00000001036.posterior.rds"
"/mnt/mfs/statgen/xqtl_workflow_testing/module/MASH/PCC_DLPFC_AC_ENSG00000001084.posterior.rds"
"/mnt/mfs/statgen/xqtl_workflow_testing/module/MASH/PCC_DLPFC_AC_ENSG00000001167.posterior.rds"
"/mnt/mfs/statgen/xqtl_workflow_testing/module/MASH/PCC_DLPFC_AC_ENSG00000001460.posterior.rds"
"/mnt/mfs/statgen/xqtl_workflow_testing/module/MASH/PCC_DLPFC_AC_ENSG00000001461.posterior.rds"
"/mnt/mfs/statgen/xqtl_workflow_testing/module/MASH/PCC_DLPFC_AC_ENSG00000001561.posterior.rds"


In [1]:
        nohup sos run pipeline/rds_to_vcf.ipynb rds_to_vcf \
            --wd ./ \
            --container //mnt/vast/hpc/csg/containers/bioinfo.sif\
            --name "Ast_Exc_Inh_Mic_OPC_Oli" \
            --analysis_units   /mnt/vast/hpc/csg/snuc_pseudo_bulk/eight_celltypes_analysis/MASH/MASH_6_celltypes/mash_output_list

nohup: ignoring input and appending output to 'nohup.out'
